# Import library

In [1]:
import sys
sys.path.append('src')
import enex_analysis as enex
from SALib.sample import saltelli
from SALib.analyze import sobol
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import dartwork_mpl as dm
import seaborn as sns
from scipy.stats import norm, uniform, triang, gamma
import warnings
from pprint import pprint
warnings.filterwarnings("ignore")
dm.use_style('dmpl_light')

Load colors...
Load colormaps...


# Primary energy use, CO2 emission, Exergy efficiency

## system setting

In [2]:
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s

EB       = enex.ElectricBoiler()
EB.r0    = 0.2
EB.H     = 0.8
EB.x_ins = 0.05
EB.dV_w_serv = MWF
EB.system_update()

GB = enex.GasBoiler()
GB.r0    = 0.12
GB.H     = 0.45
GB.x_ins = 0.05
GB.dV_w_serv = MWF
GB.system_update()

HPB = enex.HeatPumpBoiler()
HPB.r0    = 0.2
HPB.H     = 0.8
HPB.x_ins = 0.05
HPB.dV_w_serv = MWF
HPB.system_update()

print(f'Mean water flow rate: {MWF*enex.m32L/enex.s2m:.3} L/min')
print(f'Volume of water in the tank:')

Mean water flow rate: 1.11 L/min
Volume of water in the tank:


## Plot

In [3]:
# Primary energy factor
PEF_elec = 2.75 
PEF_NG = 1.1

# CO2 emission factor
Wh2MJ = 10**6/(1*enex.h2s) # enex.h2s = 3600 s/h
C_to_CO2 = 44/12 # 탄소 배출량당 이산화탄소 배출량 분자량에 의한 환산계수 
CF_NG_ori = 15.281 # tC/TJ = kgC/GJ = gC/MJ

CF_elec = 0.4747 # tCO2/MWh = gCO2/Wh
CF_NG = (CF_NG_ori*C_to_CO2)/Wh2MJ # gCO2/Wh

# Energy use
energy_use = [
        EB.E_heater,
        GB.E_NG,
        HPB.E_cmp + HPB.E_fan
]

# Exergy use
exergy_use = [
        EB.X_heater,
        GB.X_NG,
        HPB.X_cmp + HPB.X_fan
]

# Primary energy use
primary_energy_use = [
        energy_use[0] * PEF_elec,
        energy_use[1] * PEF_NG,
        energy_use[2] * PEF_elec,
]

# CO2 emission
CO2_emission = [
        energy_use[0] * CF_elec, 
        energy_use[1] * CF_NG,
        energy_use[2] * CF_elec
]

eX_efficiency = np.array([
        EB.X_eff, GB.X_eff, HPB.X_eff
]) * 100

# Create figure and axis - adjust figure size for vertical layout
nrows = 1
ncols = 4
fig, ax = plt.subplots(nrows, ncols, figsize=(dm.cm2in(16), dm.cm2in(4)), dpi=200, facecolor=None, edgecolor='k')

for ridx in range(nrows):
    for cidx in range(ncols):
        idx    = ridx * ncols + cidx
        data   = [energy_use, primary_energy_use, CO2_emission, eX_efficiency][idx]
        heading   = ['Energy use [W]', 'Primary energy use [W]', 'CO$_2$ emission [gCO$_2$/h]', 'Exergy efficiency [%]'][idx]
        labels = [['EB', 'GB', 'HPB'], # ['EB\n(E_heater)', 'GB\n(E_NG})', 'HPB\n(E_fan} + E_cmp})']
                  ['EB', 'GB', 'HPB'],
                  ['EB', 'GB', 'HPB'],
                  ['EB', 'GB', 'HPB']][idx]

        group_margin = 0.15  # margin between groups
        bar_margin   = 0.1  # margin between bars
        bar_width    = 0.25

        dN     = len(data)  # number of data
        x_pos0 = np.array([i * (bar_margin + bar_width) for i in range(dN)]) # 바 margin과 bar width를 곱해서 그룹 마진이 없는 x_pos0에 저장
        gl1    = [[1,1,1],[1,1,1],[1,1,1],[1,1,1]][idx]  # group
        gN     = len(gl1)  # number of groups
        gl2    = np.cumsum(gl1)

        x_pos = x_pos0[:gl2[0]].tolist()
        for i in range(gN - 1):  # Add group_margin between arrays
            x_pos.extend((x_pos0[gl2[i]:gl2[i+1]] + group_margin * (i + 1)).tolist()) # 그룹 마진을 추가

        # Plot parameters
        sp_ratio = 0.5
        xmar = bar_width
        xmax = max(x_pos)
        xmin = min(x_pos)
        ymar = 0
        ymax = [9000, 9000, 1500, 30][idx]
        ymin = [0, 0, 0, 0][idx]
        yint = [3000, 3000, 500, 10][idx]

        # Create vertical bars
        bars = ax[idx].bar(x_pos, data, bar_width, color=[['dm.red7', 'dm.red5','dm.red3'],
                                                          ['dm.yellow7', 'dm.yellow5','dm.yellow3'],
                                                          ['dm.lime7', 'dm.lime5','dm.lime3'],
                                                          ['dm.grape7', 'dm.grape5','dm.grape3']][idx])

        # Annotation
        ax[idx].annotate(f'{heading}', xy=(0.5, 1.1), xycoords='axes fraction', color='dm.gray9',
                        horizontalalignment='center', verticalalignment='bottom', fontsize=dm.fs(0), fontweight=400)

        # Add value labels on top of each bar
        for bidx, bar in enumerate(bars):
                height = bar.get_height()
                dc = [0,0,0,1][cidx]
                ax[idx].text(bar.get_x() + bar.get_width() / 2., height + ymax / 150,
                                f'{height:.{dc}f}', ha='center', va='bottom', fontsize=dm.fs(-2.5),fontweight=300)

            
        # Axes limits
        ax[idx].set_xlim(xmin - xmar, xmax + xmar)
        ax[idx].set_ylim(ymin - ymar, ymax + ymar)


        # Tick parameters
        ax[idx].tick_params(axis='x', direction='in', labelsize=dm.fs(-1.5),
                            which='major', length=0, width=0.5 * sp_ratio, pad=3)
        ax[idx].tick_params(axis='x', direction='in', labelsize=dm.fs(-1.5),
                            which='minor', length=0, width=0.25 * sp_ratio, pad=3)
        ax[idx].tick_params(axis='y', direction='out', labelsize=dm.fs(-1.5),
                            which='major', length=2, width=0.5 * sp_ratio, pad=3)
        ax[idx].tick_params(axis='y', direction='out', labelsize=dm.fs(-1.5),
                            which='minor', length=1, width=0.25 * sp_ratio, pad=3)

        # Labels and ticks
        ax[idx].set_xticks(x_pos)
        ax[idx].set_yticks(np.arange(ymin,ymax+yint,yint))  # No y-ticks
        ax[idx].set_xticklabels(labels, ha='center', fontsize=dm.fs(-1.5), fontweight=400)
        ax[idx].set_yticklabels(np.arange(ymin,ymax+yint,yint), fontsize=dm.fs(-1.5), fontweight=300)
        
        # Minor ticks
        # ax[idx].xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))
        # ax[idx].yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))

        # Spines
        for k in ['top','right']:  # Only keep bottom spine
            ax[idx].spines[k].set_visible(False)
        ax[0].spines['bottom'].set_linewidth(0.5 * sp_ratio)
        ax[0].spines['bottom'].set_color('k')

plt.subplots_adjust(wspace=0.3)
dm.simple_layout(fig, bbox=(0.01,1,0.0,.945), verbose=False)
img_stem = f'figure/EB_GB_HPB_overall_results'
plt.savefig(f'{img_stem}.svg', transparent=True)
plt.savefig(f'{img_stem}.png',dpi=600, transparent=True)
dm.util.save_and_show(fig)


findfont: Font family ['cursive'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'cursive' not found because none of the following families were found: Apple Chancery, Textile, Zapf Chancery, Sand, Script MT, Felipa, Comic Neue, Comic Sans MS, cursive


# Monte Carlo simulation

### 공통 변수 및 함수

In [4]:
# # ===================================================================
# # 공통 시뮬레이션 변수 범위 정의
# # ===================================================================

# # -------------------------------------------------------------------
# # 지중 조건 (Ground Conditions)
# # -------------------------------------------------------------------
# time_range = [0, 2000]        # 시간 범위 (시간 단위)
# ####################################################################################################################
# # k_g_range   = [1.5, 2.5]      # 토양 열전도율 (W/m·K)
# # c_g_range   = [700, 900]      # 토양 비열 (J/kg·K)
# # rho_g_range = [1800, 2200]    # 토양 밀도 (kg/m³)
# R_b_range   = [0.1, 0.2]    # 유효 보어홀 열저항
# T_g_range_cooling = [16.0, 24.0]    # 초기 지중 온도 (℃) 
# T_g_range_heating = [8.0, 16.0]    # 초기 지중 온도 (℃)
# #################################################################################################################################################
# # -------------------------------------------------------------------
# # 부하 및 실내 조건 (Load & Room Conditions)
# # -------------------------------------------------------------------
# # 냉방 (Cooling)
# Q_c_load_range     = [3000, 9000] # 실내 냉방 부하 (W)
# T_a_room_c_range   = [20.0, 25.0] # 실내 공기 온도 (냉방) (℃)

# # 난방 (Heating)
# Q_h_load_range     = [3000, 9000] # 실내 난방 부하 (W)
# T_a_room_h_range   = [20.0, 25.0] # 실내 공기 온도 (난방) (℃)

# # -------------------------------------------------------------------
# # 급탕 시스템 공통 변수 (Hot Water System Common Variables)
# # - 이전에 정의했던 변수들도 여기에 포함하여 관리합니다.
# # -------------------------------------------------------------------
# # 운전 조건 (부하)
# dV_w_serv_range = [0.5, 3.0]     # 탱크 온수 사용량 (L/min)
# T_w_serv_range  = [35.0, 50.0]   # 공급 온수 온도 (℃)

# # 저탕조 (설계/성능)
# T_w_tank_range  = [55.0, 65.0]   # 탱크 내 온수 온도 (℃)
# x_ins_range     = [0.03, 0.15]   # 탱크 단열 두께 (m)
# k_ins_range     = [0.025, 0.04]  # 단열재 열전도율 (W/m·K)
# h_o_range       = [10.0, 20.0]   # 종합 열전달계수 (W/m²·K)

# def supply_water_temp(T0):
#     """T_w_sup = 0.7 * T0 + 10 회귀식 임시 사용"""
#     return 0.7 * T0 + 10

# def ground_temp(T0):
#     """땅속 온도는 해당 지역의 연평균 기온에 수렴하려는 경향이 있습니다. 광주광역시의 연평균 기온이
#     약 14.5°C임을 고려할 때, 이 모델은 연중 지중 온도가 약 10°C(겨울)에서 16°C(여름) 사이에서 완만하게 변하도록 합니다.
#     y절편 12는 이러한 지역적 특성을 반영한 기준 온도입니다."""
#     return 0.2 * T0 + 12

### 수정 test

In [4]:
# ===================================================================
# 공통 시뮬레이션 변수 범위 정의
# ===================================================================

# -------------------------------------------------------------------
# 지중 조건 (Ground Conditions)
# -------------------------------------------------------------------
time_range = [0, 2000]        # 시간 범위 (시간 단위)
####################################################################################################################
# k_g_range   = [1.5, 2.5]      # 토양 열전도율 (W/m·K)
# c_g_range   = [700, 900]      # 토양 비열 (J/kg·K)
# rho_g_range = [1800, 2200]    # 토양 밀도 (kg/m³)
R_b_range   = [0.1, 0.2]    # 유효 보어홀 열저항
T_g = 15.0
#################################################################################################################################################
# -------------------------------------------------------------------
# 부하 및 실내 조건 (Load & Room Conditions)
# -------------------------------------------------------------------
# 냉방 (Cooling)
Q_c_load_range     = [3000, 9000] # 실내 냉방 부하 (W)
T_a_room_c_range   = [20.0, 25.0] # 실내 공기 온도 (냉방) (℃)

# 난방 (Heating)
Q_h_load_range     = [3000, 9000] # 실내 난방 부하 (W)
T_a_room_h_range   = [20.0, 25.0] # 실내 공기 온도 (난방) (℃)

# -------------------------------------------------------------------
# 급탕 시스템 공통 변수 (Hot Water System Common Variables)
# - 이전에 정의했던 변수들도 여기에 포함하여 관리합니다.
# -------------------------------------------------------------------
# 운전 조건 (부하)
dV_w_serv_range = [2.0, 8.5]     # 탱크 온수 사용량 (L/min)
T_w_serv_range  = [35.0, 50.0]   # 공급 온수 온도 (℃)

# 저탕조 (설계/성능)
T_w_tank_range  = [55.0, 65.0]   # 탱크 내 온수 온도 (℃)
x_ins_range     = [0.03, 0.10]   # 탱크 단열 두께 (m)
k_ins_range     = [0.025, 0.04]  # 단열재 열전도율 (W/m·K)
h_o_range       = [10.0, 20.0]   # 종합 열전달계수 (W/m²·K)

def supply_water_temp(T0):
    """T_w_sup = 0.7 * T0 + 10 회귀식 임시 사용"""
    return 0.7 * T0 + 10

def ground_temp(T0):
    """땅속 온도는 해당 지역의 연평균 기온에 수렴하려는 경향이 있습니다. 광주광역시의 연평균 기온이
    약 14.5°C임을 고려할 때, 이 모델은 연중 지중 온도가 약 10°C(겨울)에서 16°C(여름) 사이에서 완만하게 변하도록 합니다.
    y절편 12는 이러한 지역적 특성을 반영한 기준 온도입니다."""
    return 0.2 * T0 + 12

## Hot water system

### EB

In [5]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의 (정의된 변수 활용)
item = {
    'names': [
        'dV_w_serv', # 탱크 온수 사용량 (L/min)
        'T_w_serv',  # 공급 온수 온도 (℃)
        'T_w_tank',  # 탱크 내 온수 온도 (℃)
        'x_ins',     # 탱크 단열 두께 (m)
        'k_ins',     # 단열재 열전도율 (W/m·K)
        'h_o'        # 종합 열전달계수 (W/m²·K)
    ],
    'bounds': [
        dV_w_serv_range,
        T_w_serv_range,
        T_w_tank_range,
        x_ins_range,
        k_ins_range,
        h_o_range
    ]
}

problem = {
    'num_vars': len(item["names"]),
    'names': item["names"],
    'bounds': item["bounds"],
    'dists': ['unif'] * len(item["names"])
}


# Saltelli 방법을 사용하여 샘플 생성
param_values = saltelli.sample(problem, num_simulations, calc_second_order=True)

# 엑서지 효율 저장 리스트
ex_eff_EB = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):
    # enex.ElectricBoiler 객체 생성
    EB = enex.ElectricBoiler()

    # 고정 파라미터 설정
    EB.T_0 = 0.0      # 환경 온도 0℃로 고정
    EB.T_w_sup = 10.0 # 상수도 공급수 온도 10℃로 고정

    # 샘플링된 파라미터 적용
    EB.dV_w_serv = param_values[i, 0]
    EB.T_w_serv  = param_values[i, 1]
    EB.T_w_tank  = param_values[i, 2]
    EB.x_ins     = param_values[i, 3]
    EB.k_ins     = param_values[i, 4]
    EB.h_o       = param_values[i, 5]

    EB.T_r_tank = EB.T_w_tank + 5  # 탱크 내 온수 온도 (℃)

    # 시스템 업데이트 및 결과 저장
    EB.system_update()
    ex_eff_EB.append(EB.X_eff * 100)

# 예시 데이터프레임 생성
df = pd.DataFrame(ex_eff_EB)
csv_path = "data/ex_eff_EB.csv"
df.to_csv(csv_path, index=False)

### GB

In [6]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의 (정의된 변수 활용)
item = {
    'names': [
        'dV_w_serv', # 탱크 온수 사용량 (L/min)
        'T_w_serv',  # 공급 온수 온도 (℃)
        'T_w_tank',  # 탱크 내 온수 온도 (℃)
        'x_ins',     # 탱크 단열 두께 (m)
        'k_ins',     # 단열재 열전도율 (W/m·K)
        'h_o',       # 종합 열전달계수 (W/m²·K)
        'eta_comb'   # 연소 챔버 효율 (가스보일러의 경우) [%]
    ],
    'bounds': [
        dV_w_serv_range,
        T_w_serv_range,
        T_w_tank_range,
        x_ins_range,
        k_ins_range,
        h_o_range,
        [0.8, 0.95]  # 연소 챔버 효율 범위 (가스보일러의 경우) [%]
    ]
}

problem = {
    'num_vars': len(item["names"]),
    'names': item["names"],
    'bounds': item["bounds"],
    'dists': ['unif'] * len(item["names"])
}


# Saltelli 방법을 사용하여 샘플 생성
param_values = saltelli.sample(problem, num_simulations, calc_second_order=True)

# 엑서지 효율 저장 리스트
ex_eff_GB = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):
    # enex.ElectricBoiler 객체 생성
    GB = enex.ElectricBoiler()

    # 고정 파라미터 설정
    GB.T_0 = 0.0      # 환경 온도 0℃로 고정
    GB.T_w_sup = 10.0 # 상수도 공급수 온도 10℃로 고정

    # 샘플링된 파라미터 적용
    GB.dV_w_serv = param_values[i, 0]
    GB.T_w_serv  = param_values[i, 1]
    GB.T_w_tank  = param_values[i, 2]
    GB.x_ins     = param_values[i, 3]
    GB.k_ins     = param_values[i, 4]
    GB.h_o       = param_values[i, 5]
    GB.eta_comb  = param_values[i, 6]  # 연소 챔버 효율 (가스보일러의 경우)
    
    GB.T_r_tank = GB.T_w_tank + 5  # 탱크 내 온수 온도 (℃)
    
    # 시스템 업데이트 및 결과 저장
    GB.system_update()
    ex_eff_GB.append(GB.X_eff * 100)
    
# 예시 데이터프레임 생성
df = pd.DataFrame(ex_eff_GB)
csv_path = "data/ex_eff_GB.csv"
df.to_csv(csv_path, index=False)


### HPB

In [7]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의 (정의된 변수 활용)
item = {
    'names': [
        'COP',    # 성능계수
        'dV_w_serv', # 탱크 온수 사용량 (L/min)
        'T_w_serv',  # 공급 온수 온도 (℃)
        'T_w_tank',  # 탱크 내 온수 온도 (℃)
        'x_ins',     # 탱크 단열 두께 (m)
        'k_ins',     # 단열재 열전도율 (W/m·K)
        'h_o'        # 종합 열전달계수 (W/m²·K)
    ],
    'bounds': [
        [2.0, 3.0],
        dV_w_serv_range,
        T_w_serv_range,
        T_w_tank_range,
        x_ins_range,
        k_ins_range,
        h_o_range
    ]
}

problem = {
    'num_vars': len(item["names"]),
    'names': item["names"],
    'bounds': item["bounds"],
    'dists': ['unif'] * len(item["names"])
}


# Saltelli 방법을 사용하여 샘플 생성
param_values = saltelli.sample(problem, num_simulations, calc_second_order=True)

# 엑서지 효율 저장 리스트
ex_eff_HPB = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):
    # enex.HeatPumpBoiler 객체 생성
    HPB = enex.HeatPumpBoiler()

    # 고정 파라미터 설정
    HPB.T0 = 0.0      # 환경 온도 0℃로 고정
    HPB.T_w_sup = 10.0 # 상수도 공급수 온도 10℃로 고정

    # 샘플링된 파라미터 적용
    HPB.COP     = param_values[i, 0]
    HPB.dV_w_serv  = param_values[i, 1]
    HPB.T_w_serv   = param_values[i, 2]
    HPB.T_w_tank   = param_values[i, 3]
    HPB.x_ins      = param_values[i, 4]
    HPB.k_ins      = param_values[i, 5]
    HPB.h_o        = param_values[i, 6]
    
    
    # 종속변수 설정
    HPB.T_r_tank = param_values[i, 3] + 5  # 탱크 내 온수 온도 (℃)
    HPB.T_r_ext = 0.0 - 10.0  # 외부 온도 (℃)
    HPB.T_a_ext_out = 0.0 - 5.0  # 외부 온도 (℃)
    
    # 시스템 업데이트 및 결과 저장
    HPB.system_update()
    ex_eff_HPB.append(HPB.X_eff * 100)

print(f'ex_eff_HPB: {np.max(ex_eff_HPB):.2f}')

# 예시 데이터프레임 생성
df = pd.DataFrame(ex_eff_HPB)
csv_path = "data/ex_eff_HPB.csv"
df.to_csv(csv_path, index=False)


ex_eff_HPB: 27.69


### SAGB

In [8]:

# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의
item = {
    'names': [
        'eta_comb',  # 보일러 연소 효율
        'dV_w_serv', # 탱크 온수 사용량 (L/min)
        'T_w_serv',  # 공급 온수 온도 (℃)
        'A_stc',
        'h_o'        # 종합 열전달계수 (W/m²·K)
    ],
    'bounds': [
        [0.80, 0.95],
        dV_w_serv_range,
        T_w_serv_range,
        [1.5, 6.0],
        h_o_range
    ]
}

problem = {
    'num_vars': len(item["names"]),
    'names': item["names"],
    'bounds': item["bounds"],
    'dists': ['unif'] * len(item["names"])
}


# Saltelli 방법을 사용하여 샘플 생성
param_values = saltelli.sample(problem, num_simulations, calc_second_order=True)

# 엑서지 효율 저장 리스트
ex_eff_SAGB = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):
    # enex.SolarAssistedGasBoiler 객체 생성
    SAGB = enex.SolarAssistedGasBoiler()

    # 고정 파라미터 설정
    SAGB.T0 = 0.0         # 환경 온도 0℃로 고정
    SAGB.T_w_sup = 10.0   # 상수도 공급수 온도 10℃로 고정
    SAGB.I_DN = 600

    # 샘플링된 파라미터 적용
    SAGB.eta_comb   = param_values[i, 0]
    SAGB.dV_w_serv  = param_values[i, 1]
    SAGB.T_w_serv   = param_values[i, 2]
    SAGB.A_stc      = param_values[i, 3]
    SAGB.h_o        = param_values[i, 4]
    
    # 시스템 업데이트 및 결과 저장
    SAGB.system_update()
    ex_eff_SAGB.append(SAGB.X_eff * 100)
    
# 예시 데이터프레임 생성
df = pd.DataFrame(ex_eff_SAGB)
csv_path = "data/ex_eff_SAGB.csv"
print(f"Exergy efficiency range: {min(ex_eff_SAGB):.2f} ~ {max(ex_eff_SAGB):.2f}")
df.to_csv(csv_path, index=False)


Exergy efficiency range: 7.50 ~ 18.39


### GSHPB

In [9]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의 (정의된 변수 활용)
item = {
    'names': [
        'time',      # 운전 시간 (h)
        'R_b',      # 유효 보어홀 열저항 (K·m/W)
        'dV_w_serv', # 탱크 온수 사용량 (L/min)
        'T_w_serv',  # 공급 온수 온도 (℃)
        'T_w_tank',  # 탱크 내 온수 온도 (℃)
        'x_ins',     # 탱크 단열 두께 (m)
        'k_ins',     # 단열재 열전도율 (W/m·K)
        'h_o'        # 종합 열전달계수 (W/m²·K)
    ],
    'bounds': [
        time_range,
        R_b_range,
        dV_w_serv_range,
        T_w_serv_range,
        T_w_tank_range,
        x_ins_range,
        k_ins_range,
        h_o_range
    ]
}

problem = {
    'num_vars': len(item["names"]),
    'names': item["names"],
    'bounds': item["bounds"],
    'dists': ['unif'] * len(item["names"])
}


# Saltelli 방법을 사용하여 샘플 생성
param_values = saltelli.sample(problem, num_simulations, calc_second_order=True)

# 엑서지 효율 저장 리스트
ex_eff_GSHPB = []
cop_GSHPB = [] 
# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):
    # enex.GroundSourceHeatPumpBoiler 객체 생성
    GSHPB = enex.GroundSourceHeatPumpBoiler()

    # 고정 파라미터 설정
    GSHPB.T0 = 0.0      # 환경 온도 0℃로 고정
    GSHPB.T_g = T_g # 지중 온도 10℃로 고정
    GSHPB.T_w_sup = 10.0 # 상수도 공급수 온도 10℃로 고정

    # 샘플링된 파라미터 적용
    GSHPB.time       = param_values[i, 0]
    GSHPB.R_b        = param_values[i, 1]
    GSHPB.dV_w_serv  = param_values[i, 2]
    GSHPB.T_w_serv   = param_values[i, 3]
    GSHPB.T_w_tank   = param_values[i, 4]
    GSHPB.x_ins      = param_values[i, 5]
    GSHPB.k_ins      = param_values[i, 6]
    GSHPB.h_o        = param_values[i, 7]

    # 종속변수 설정
    GSHPB.T_r_tank = GSHPB.T_w_tank + 5

    # 시스템 업데이트 및 결과 저장
    GSHPB.system_update()
    ex_eff_GSHPB.append(GSHPB.X_eff * 100)
    cop_GSHPB.append(GSHPB.COP)

# 예시 데이터프레임 생성
df = pd.DataFrame(ex_eff_GSHPB)
csv_path = "data/ex_eff_GSHPB.csv"
df.to_csv(csv_path, index=False)
print(f"Exergy efficiency range: {min(ex_eff_GSHPB):.2f} ~ {max(ex_eff_GSHPB):.2f}")
print(f'min COP: {min(cop_GSHPB):.2f}, max COP: {max(cop_GSHPB):.2f}')

Exergy efficiency range: 18.48 ~ 36.91
min COP: 1.94, max COP: 4.33


### ALL

In [10]:
ex_eff_EB = pd.read_csv("data/ex_eff_EB.csv")
ex_eff_GB = pd.read_csv("data/ex_eff_GB.csv")
HPB_data = pd.read_csv("data/ex_eff_HPB.csv")
SAGB_data = pd.read_csv("data/ex_eff_SAGB.csv")
GSHPB_data = pd.read_csv("data/ex_eff_GSHPB.csv")

ex_eff_EB = np.array(ex_eff_EB).flatten()
ex_eff_GB = np.array(ex_eff_GB).flatten()
ex_eff_HPB = np.array(HPB_data).flatten()
ex_eff_SAGB = np.array(SAGB_data).flatten()
ex_eff_GSHPB = np.array(GSHPB_data).flatten() 

# 각 시스템별 exergy efficiency min/max 계산
systems = [
    ('Ground source heat pump boiler', ex_eff_GSHPB, 'tw.blue:400'),
    ('Solar-assisted gas boiler', ex_eff_SAGB, 'tw.green:400'),
    ('Heat pump boiler', ex_eff_HPB, 'tw.indigo:400'),
    ('Gas boiler', ex_eff_GB, 'tw.pink:400'),
    ('Electric boiler', ex_eff_EB, 'tw.orange:400'),
]

xmin, xmax, xint, xmar = 0, 50, 5, 0
ymin, ymax, yint, ymar = 0, 1.0, 0.2, 0

# 효율 등급 구간 (start, end, color, label)
E = 10
D = 15
C = 20
B = 25
A = 30
A_plus = 50

eff_zones = [
    (0, E, '#E74C3C', f'E\n0~{E}%'),
    (E, D, '#FF8C00', f'D\n{E}~{D}%'),
    (D, C, '#FFD700', f'C\n{D}~{C}%'),
    (C, B, '#90EE90', f'B\n{C}~{B}%'),
    (B, A, '#32CD32', f'A\n{B}~{A}%'),
    (A, A_plus, '#228B22', f'A+\n{A}~{A_plus}%')
]

fig, ax = plt.subplots(figsize=(dm.cm2in(12), dm.cm2in(4.5)), dpi=600)

# 등급 구간 색상 채우기 및 등급 기호 표시
for start, end, color, grade in eff_zones:
    ax.axvspan(start, end, color=color, alpha=0.3, zorder=0)
    # 상단 중앙에 등급 기호 표시
    ax.text((start + end) / 2, len(systems) + 0.7, grade, ha='center', va='bottom',
            fontsize=dm.fs(-0.5), fontweight=700, color=color, zorder=10)

for i, (label, eff_list, color) in enumerate(systems, 1):
    min_eff = np.min(eff_list)
    max_eff = np.max(eff_list)
    mid_eff = 0.5 * (min_eff + max_eff)
    ax.hlines(i, min_eff, max_eff, colors=color, linewidth=10)
    ax.text(mid_eff, i, label, ha='center', va='center', fontsize=dm.fs(-1.5), color='black')

ax.set_yticks([])
ax.set_ylabel('')
ax.set_ylim(0.5, len(systems) + 0.7)
ax.set_xlabel("Exergy efficiency [%]", fontsize=dm.fs(0), fontweight=300)
ax.set_xlim(xmin, xmax)
ax.set_xticks(np.arange(xmin, xmax + xint*0.9, xint))
ax.spines['left'].set_visible(False)
plt.tight_layout()
folder_path ='figure/monte_carlo'
fig_name = 'All_DHW_ex_eff_grade'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)
fig.savefig(f'{folder_path}/{fig_name}.svg', transparent=True)
# dm.simple_layout(fig, bbox=(0, 1, 0.03, 1.05), verbose=False)
dm.util.save_and_show(fig)
plt.close()

## Cooling

### ASHP

In [11]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의
item = {
    'names': [
        'Q_r_int',   # 실내 냉방 부하 (W)
        'T_a_room'   # 실내 공기 온도 (℃)
    ],
    'bounds': [
        Q_c_load_range,
        T_a_room_c_range,
    ]
}

problem = {
    'num_vars': len(item["names"]),
    'names': item["names"],
    'bounds': item["bounds"],
    'dists': ['unif'] * len(item["names"])
}


# Saltelli 방법을 사용하여 샘플 생성
param_values = saltelli.sample(problem, num_simulations, calc_second_order=True)

# 엑서지 효율 저장 리스트
ex_eff_ASHPC = []
cop_ASHPC = []
# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):
    # enex.AirSourceHeatPump_cooling 객체 생성
    ASHPC = enex.AirSourceHeatPump_cooling()

    # 고정 파라미터 설정
    ASHPC.T0 = 32.0      # 환경 온도 30℃로 고정
    ASHPC.Q_r_max = 9000  # 최대 냉방 부하 9000W로 고정

    # 샘플링된 파라미터 적용
    ASHPC.Q_r_int   = param_values[i, 0]
    ASHPC.T_a_room  = param_values[i, 1]
    
    # 종속변수 설정
    ASHPC.T_r_ext = ASHPC.T0 + 15.0  # 외부 온도 (℃)
    ASHPC.T_a_ext_out = ASHPC.T0 + 10.0  # 외부 온도 (℃)
    
    ASHPC.T_r_int = ASHPC.T_a_room - 10.0  # 실내 온도 (℃)
    ASHPC.T_a_int_out = ASHPC.T_a_room - 5.0  # 실내 온도 (℃)
    
    
    # 시스템 업데이트 및 결과 저장
    ASHPC.system_update()
    ex_eff_ASHPC.append(ASHPC.X_eff * 100)
    cop_ASHPC.append(ASHPC.COP)

# 예시 데이터프레임 생성
df = pd.DataFrame(ex_eff_ASHPC)
csv_path = "data/ex_eff_ASHPC.csv"
df.to_csv(csv_path, index=False)
print(f"Exergy efficiency range: {min(ex_eff_ASHPC):.2f} ~ {max(ex_eff_ASHPC):.2f}")
print(f'min COP: {min(cop_ASHPC):.2f}, max COP: {max(cop_ASHPC):.2f}')

Exergy efficiency range: 7.41 ~ 15.87
min COP: 2.92, max COP: 4.09


### GSHP

In [12]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의
item = {
    'names': [
        'time',      # 운전 시간 (h)
        'Q_r_int',   # 실내 냉방 부하 (W)
        'T_a_room',  # 실내 공기 온도 (℃)
        'R_b',      # 유효 보어홀 열저항 (K·m/W)
    ],
    'bounds': [
        time_range,
        Q_c_load_range,
        T_a_room_c_range,
        R_b_range,
    ]
}

problem = {
    'num_vars': len(item["names"]),
    'names': item["names"],
    'bounds': item["bounds"],
    'dists': ['unif'] * len(item["names"])
}


# Saltelli 방법을 사용하여 샘플 생성
param_values = saltelli.sample(problem, num_simulations, calc_second_order=True)

# 엑서지 효율 저장 리스트
ex_eff_GSHPC = []
cop_GSHPC = []
# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):
    # enex.GroundSourceHeatPump_cooling 객체 생성
    GSHPC = enex.GroundSourceHeatPump_cooling()

    # 고정 파라미터 설정
    GSHPC.T0 = 32.0      # 환경 온도 32℃로 고정
    GSHPC.T_g = T_g     # 지중 온도 21℃로 고정

    # 샘플링된 파라미터 적용
    GSHPC.time      = param_values[i, 0]
    GSHPC.Q_r_int   = param_values[i, 1]
    GSHPC.T_a_room  = param_values[i, 2]
    GSHPC.R_b       = param_values[i, 3]
    
    # 종속변수 설정
    GSHPC.T_r_int = GSHPC.T_a_room - 10.0  # 실내 온도 (℃)
    GSHPC.T_a_int_out = GSHPC.T_a_room - 5.0  # 실내 온도 (℃)

    # 시스템 업데이트 및 결과 저장
    GSHPC.system_update()
    ex_eff_GSHPC.append(GSHPC.X_eff * 100)
    cop_GSHPC.append(GSHPC.COP)
    
# 예시 데이터프레임 생성
df = pd.DataFrame(ex_eff_GSHPC)
csv_path = "data/ex_eff_GSHPC.csv"
df.to_csv(csv_path, index=False)
print(f'min COP: {min(cop_GSHPC):.2f}, max COP: {max(cop_GSHPC):.2f}')
print(f"Exergy efficiency range: {min(ex_eff_GSHPC):.2f} ~ {max(ex_eff_GSHPC):.2f}")

min COP: 6.28, max COP: 28.16
Exergy efficiency range: 18.88 ~ 29.53


### ALL

In [13]:
# 데이터는 이미 ex_eff_ASHPC, ex_eff_GSHPC 변수에 numpy array로 존재함
ex_eff_ASHPC = pd.read_csv("data/ex_eff_ASHPC.csv")
ex_eff_GSHPC = pd.read_csv("data/ex_eff_GSHPC.csv")

ex_eff_ASHPC = np.array(ex_eff_ASHPC).flatten()
ex_eff_GSHPC = np.array(ex_eff_GSHPC).flatten()

# 효율 등급 구간 (start, end, color, label)
E = 5
D = 10
C = 15
B = 20
A = 25
A_plus = 50

eff_zones = [
    (0, E, '#E74C3C', f'E\n0~{E}%'),
    (E, D, '#FF8C00', f'D\n{E}~{D}%'),
    (D, C, '#FFD700', f'C\n{D}~{C}%'),
    (C, B, '#90EE90', f'B\n{C}~{B}%'),
    (B, A, '#32CD32', f'A\n{B}~{A}%'),
    (A, A_plus, '#228B22', f'A+\n{A}~{A_plus}%')
]

systems = [
    ('Ground source heat pump', ex_eff_GSHPC, 'tw.emerald:400'),
    ('Air source heat pump', ex_eff_ASHPC, 'tw.indigo:400'),
]

xmin, xmax, xint, xmar = 0, 50, 5, 0

fig, ax = plt.subplots(figsize=(dm.cm2in(12), dm.cm2in(3.5)), dpi=600)

# 등급 구간 색상 채우기 및 등급 기호 표시
for start, end, color, grade in eff_zones:
    ax.axvspan(start, end, color=color, alpha=0.3, zorder=0)
    ax.text((start + end) / 2, len(systems) + 0.7, grade, ha='center', va='bottom',
            fontsize=dm.fs(-0.5), fontweight=700, color=color, zorder=10)

for i, (label, eff_list, color) in enumerate(systems, 1):
    min_eff = np.min(eff_list)
    max_eff = np.max(eff_list)
    mid_eff = 0.5 * (min_eff + max_eff)
    ax.hlines(i, min_eff, max_eff, colors=color, linewidth=10)
    ax.text(mid_eff, i, label, ha='center', va='center', fontsize=dm.fs(-1.5), color='black')

ax.set_yticks([])
ax.set_ylabel('')
ax.set_ylim(0.5, len(systems) + 0.7)
ax.set_xlabel("Exergy efficiency [%]", fontsize=dm.fs(0), fontweight=300)
ax.set_xlim(xmin, xmax)
ax.set_xticks(np.arange(xmin, xmax + xint*0.9, xint))
ax.spines['left'].set_visible(False)
plt.tight_layout()
fig_name = 'All_Cooling_ex_eff_grade'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)
fig.savefig(f'{folder_path}/{fig_name}.svg', transparent=True)
# dm.simple_layout(fig, bbox=(0, 1, 0.13, 1.05), verbose=False)
dm.util.save_and_show(fig)
plt.close()


## Heating

### ASHP

In [14]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의 (공통 변수 사용)
item = {
    'names': [
        'Q_r_int',   # 실내 난방 부하 (W)
        'T_a_room'   # 실내 공기 온도 (℃)
    ],
    'bounds': [
        Q_h_load_range,   # 공통 변수 사용
        T_a_room_h_range  # 공통 변수 사용
    ]
}

problem = {
    'num_vars': len(item["names"]),
    'names': item["names"],
    'bounds': item["bounds"],
    'dists': ['unif'] * len(item["names"])
}


# Saltelli 방법을 사용하여 샘플 생성
param_values = saltelli.sample(problem, num_simulations, calc_second_order=True)

# 엑서지 효율 저장 리스트
ex_eff_ASHPH = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):
    # enex.AirSourceHeatPump_heating 객체 생성
    ASHPH = enex.AirSourceHeatPump_heating()

    # 고정 파라미터 설정
    ASHPH.T0 = 0.0      # 환경 온도 0℃로 고정
    
    ASHPH.Q_r_max = 9000.0  # 최대 난방 부하 9000W로 고정

    # 샘플링된 파라미터 적용
    ASHPH.Q_r_int   = param_values[i, 0]
    ASHPH.T_a_room  = param_values[i, 1]
    
    # 종속 변수 계산 및 설정
    ASHPH.T_r_int = ASHPH.T_a_room + 15.0
    ASHPH.T_a_int_out = ASHPH.T_a_room + 10.0
    
    ASHPH.T_r_ext = ASHPH.T0 - 10.0
    ASHPH.T_a_ext_out = ASHPH.T0 - 5.0
    
    # 시스템 업데이트 및 결과 저장
    ASHPH.system_update()
    ex_eff_ASHPH.append(ASHPH.X_eff * 100)
# 예시 데이터프레임 생성
df = pd.DataFrame(ex_eff_ASHPH)
csv_path = "data/ex_eff_ASHPH.csv"
df.to_csv(csv_path, index=False)


### GSHP

In [15]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의
item = {
    'names': [
        'time',      # 운전 시간 (h)
        'Q_r_int',   # 실내 난방 부하 (W)
        'T_a_room',  # 실내 공기 온도 (℃)
        'R_b'      # 유효 보어홀 열저항 (K·m/W)

    ],
    'bounds': [
        time_range,      # 공통 변수 사용  
        Q_h_load_range,   # 공통 변수 사용
        T_a_room_h_range, # 공통 변수 사용
        R_b_range,        # 유효 보어홀 열저항 범위
    ]
}

problem = {
    'num_vars': len(item["names"]),
    'names': item["names"],
    'bounds': item["bounds"],
    'dists': ['unif'] * len(item["names"])
}


# Saltelli 방법을 사용하여 샘플 생성
param_values = saltelli.sample(problem, num_simulations, calc_second_order=True)

# 엑서지 효율 저장 리스트
ex_eff_GSHPH = []
cop_GSHPH = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):
    # enex.GroundSourceHeatPump_heating 객체 생성
    GSHPH = enex.GroundSourceHeatPump_heating()

    # 고정 파라미터 설정
    GSHPH.T0 = 0.0       # 환경 온도 0℃로 고정
    GSHPH.T_g = 15.0     # 지중 온도 11℃로 고정

    # 샘플링된 파라미터 적용
    GSHPH.time      = param_values[i, 0]
    GSHPH.Q_r_int   = param_values[i, 1]
    GSHPH.T_a_room  = param_values[i, 2]
    GSHPH.R_b      = param_values[i, 3]

    # 종속 변수 계산 및 설정
    GSHPH.T_a_int_out = GSHPH.T_a_room + 10.0
    GSHPH.T_r_int     = GSHPH.T_a_room + 15.0

    # 시스템 업데이트 및 결과 저장
    GSHPH.system_update()
    ex_eff_GSHPH.append(GSHPH.X_eff * 100)
    cop_GSHPH.append(GSHPH.COP)

# 예시 데이터프레임 생성
df = pd.DataFrame(ex_eff_GSHPH)
csv_path = "data/ex_eff_GSHPH.csv"
df.to_csv(csv_path, index=False)
print(f'min COP: {min(cop_GSHPH):.2f}, max COP: {max(cop_GSHPH):.2f}')
print(f"Exergy efficiency range: {min(ex_eff_GSHPH):.2f} ~ {max(ex_eff_GSHPH):.2f}")

min COP: 3.18, max COP: 6.51
Exergy efficiency range: 23.94 ~ 40.42


### EH

In [16]:
# 시뮬레이션 반복 횟수
num_simulations = 10000

# 문제 정의
item = {
    'names': [
        'T_a_room',  # 실내 공기 온도 (℃)
        # 'T_mr',      # 실내 평균 복사 온도 (℃)
    ],
    'bounds': [
        T_a_room_h_range, # 공통 변수 사용
        # [18.0, 23.0],
    ]
}

problem = {
    'num_vars': len(item["names"]),
    'names': item["names"],
    'bounds': item["bounds"],
    'dists': ['unif'] * len(item["names"])
}


# Saltelli 방법을 사용하여 샘플 생성
param_values = saltelli.sample(problem, num_simulations, calc_second_order=True)

# 엑서지 효율 저장 리스트
ex_eff_EH = []
surf_temp_EH = []

# 몬테카를로 시뮬레이션 및 엑서지 효율 계산
for i in range(param_values.shape[0]):
    # enex.ElectricHeater 객체 생성
    EH = enex.ElectricHeater()
    EH.T0 = 0.0      # 환경 온도 0℃로 고정

    # 고정 파라미터 설정
    EH.dt = 3600 

    # 샘플링된 파라미터 적용
    EH.T_a_room  = param_values[i, 0]
    # EH.T_mr      = param_values[i, 1]

    # 시스템 업데이트 및 결과 저장
    EH.system_update()
    
    ex_eff_EH.append(EH.X_eff * 100)
    surf_temp_EH.append(EH.T_hs)      

# 예시 데이터프레임 생성
df = pd.DataFrame(ex_eff_EH)
csv_path = "data/ex_eff_EH.csv"
df.to_csv(csv_path, index=False)
print(f"Exergy efficiency range: {min(ex_eff_EH):.2f} ~ {max(ex_eff_EH):.2f}")
print(f"Surface temperature range: {min(surf_temp_EH):.2f} ~ {max(surf_temp_EH):.2f}")

Exergy efficiency range: 12.54 ~ 12.88
Surface temperature range: 339.07 ~ 340.91


In [18]:
340-273

67

### ALL

In [17]:
ex_eff_ASHPH = pd.read_csv("data/ex_eff_ASHPH.csv")
ex_eff_GSHPH = pd.read_csv("data/ex_eff_GSHPH.csv")
ex_eff_EH = pd.read_csv("data/ex_eff_EH.csv")

ex_eff_ASHPH = np.array(ex_eff_ASHPH).flatten()
ex_eff_GSHPH = np.array(ex_eff_GSHPH).flatten()
ex_eff_EH = np.array(ex_eff_EH).flatten()

# 각 시스템별 exergy efficiency min/max 계산
systems = [
    ('Electric heater', ex_eff_EH, 'tw.green:400'),
    ('Ground source heat pump boiler', ex_eff_GSHPH, 'tw.yellow:400'),
    ('Air source heat pump boiler', ex_eff_ASHPH, 'tw.red:400'),
]

xmin, xmax, xint, xmar = 0, 50, 5, 0
ymin, ymax, yint, ymar = 0, 1.0, 0.2, 0

# 효율 등급 구간 (start, end, color, label)
E = 15
D = 20
C = 25
B = 30
A = 35
A_plus = 50
eff_zones = [
    (0, E, '#E74C3C', f'E\n0~{E}%'),
    (E, D, '#FF8C00', f'D\n{E}~{D}%'),
    (D, C, '#FFD700', f'C\n{D}~{C}%'),
    (C, B, '#90EE90', f'B\n{C}~{B}%'),
    (B, A, '#32CD32', f'A\n{B}~{A}%'),
    (A, A_plus, '#228B22', f'A+\n{A}~{A_plus}%')
]

xmin, xmax, xint, xmar = 0, 50, 5, 0

fig, ax = plt.subplots(figsize=(dm.cm2in(12), dm.cm2in(3.8)), dpi=600)

# 등급 구간 색상 채우기 및 등급 기호 표시
for start, end, color, grade in eff_zones:
    ax.axvspan(start, end, color=color, alpha=0.3, zorder=0)
    ax.text((start + end) / 2, len(systems) + 0.7, grade, ha='center', va='bottom',
            fontsize=dm.fs(-0.5), fontweight=700, color=color, zorder=10)

for i, (label, eff_list, color) in enumerate(systems, 1):
    min_eff = np.min(eff_list)
    max_eff = np.max(eff_list)
    mid_eff = 0.5 * (min_eff + max_eff)
    ax.hlines(i, min_eff, max_eff, colors=color, linewidth=10)
    ax.text(mid_eff, i, label, ha='center', va='center', fontsize=dm.fs(-1.5), color='black')

ax.set_yticks([])
ax.set_ylabel('')
ax.set_ylim(0.5, len(systems) + 0.7)
ax.set_xlabel("Exergy efficiency [%]", fontsize=dm.fs(0), fontweight=300)
ax.set_xlim(xmin, xmax)
ax.set_xticks(np.arange(xmin, xmax + xint*0.9, xint))
ax.spines['left'].set_visible(False)
plt.tight_layout()
fig_name = 'All_Heating_ex_eff_grade'
fig.savefig(f'{folder_path}/{fig_name}.png', dpi=600)
fig.savefig(f'{folder_path}/{fig_name}.svg', transparent=True)
# dm.simple_layout(fig, bbox=(0, 1, 0.13, 0.95), verbose=False)
dm.util.save_and_show(fig)
plt.close()

# Water fall chart

## EB

In [8]:
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s
print(f'Mean water flow rate: {MWF*enex.m32L/enex.s2m:.3} L/min')

EB       = enex.ElectricBoiler()
EB.k_shell = 25
EB.r0    = 0.2
EB.H     = 0.8
EB.x_ins = 0.05
EB.dV_w_serv = MWF
EB.system_update()

# ----------------------------
# 데이터 정의 (Energy, Entropy, Exergy 각각 Tank + Mixing 포함)
# ----------------------------
# 데이터 정의 및 딕셔너리 구성
Energy = {
    "components": [
        {"value": EB.Q_w_sup_tank, "sign": 1, "label": 'Supply water\nto tank'},
        {"value": EB.E_heater, "sign": 1, "label": 'Electricity input\nto heater'},
        {"value": 0, "sign":  1, "label": ''},
        {"value": EB.Q_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": EB.Q_w_sup_mix, "sign":  1, "label": 'Supply water\nto mixing valve'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": EB.Q_w_serv, "sign": 1, "label": 'Served\nhot water'} 
    ],
    "color" : 'tw.lime:',
    "ylabel": 'Energy [W]',
    "ymax"   : 3600,
    "yint"   : 900
}
    
Entropy = {
    "components": [
        {"value": EB.S_w_sup_tank, "sign": 1, "label": 'Supply water\nto tank'},
        {"value": EB.S_heater, "sign": 1, "label": 'Electricity input\nto heater'},
        {"value": EB.S_g_tank, "sign": 1, "label": 'Consumption\nin tank'},
        {"value": EB.S_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": EB.S_w_sup_mix, "sign": 1, "label": 'Supply water\nto mixing valve'},
        {"value": EB.S_g_mix, "sign": 1, "label": 'generated (mix)'},
        {"value": EB.S_w_serv, "sign": 1, "label": 'Served\nhot water'}
    ],
    "color" : 'tw.amber:',
    "ylabel": 'Entropy [W/K]',
    "ymax"   : 16,
    "yint"   : 4
}
    
Exergy = {
    "components": [
        {"value": EB.X_w_sup_tank, "sign": 1, "label": 'Supply water\nto tank'},
        {"value": EB.X_heater, "sign": 1, "label": 'Electricity input\nto heater'},
        {"value": EB.X_c_tank, "sign": -1, "label": 'Consumption\nin tank'},
        {"value": EB.X_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": EB.X_w_sup_mix, "sign": 1, "label": 'Supply water\nto mixing valve'},
        {"value": EB.X_c_mix, "sign": -1, "label": 'Consumption\nin mixing valve'},
        {"value": EB.X_w_serv, "sign": 1, "label": 'Served\nhot water'}
    ],
    "color" : 'tw.purple:',
    "ylabel":  'Exergy [W]',
    "ymax"   : 3600,
    "yint"   : 900
}

data_list = [Energy, Exergy]

# Figure 구성
nrows = 3
ncols = 1

fig, axs = plt.subplots(2, 1, figsize=(dm.cm2in(14), dm.cm2in(8)), dpi=200)

for i, data in enumerate(data_list):
    
    ax = axs[i]
    
    # Components
    components = data["components"]
    value      = np.array([comp["value"] for comp in components])
    sign       = np.array([comp["sign"] for comp in components])
    labels     = [comp["label"] for comp in components]
    signed     = value * sign
    color      = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.05  
    bm = DL * 0.06 
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops    = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1]  = last_height

    # bar plot
    bars = ax.bar(
        x_pos, signed, width=bw, bottom=bottoms,
        color=[color + '500' if h > 0 else color + '100' for h in signed],
        edgecolor=[color + '800' if h > 0 else color + '500' for h in signed],
        linewidth=0.5
    )
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        text_y = bottoms[j] + h + offset if h >= 0 else tops[j] - h + offset
        ax.text(bar.get_x() + bw/2, text_y, '' if h == 0 else f'{h:.0f}',
            ha='center', va='bottom', fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-1))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize = dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(min(0, min(value)*1.1), data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(0), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.065, 0.5)
    
# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=(0, 1, 0, 1))
plt.savefig('figure/EB_waterfall.svg')
plt.savefig('figure/EB_waterfall.png', dpi=600)
dm.util.save_and_show(fig)

Mean water flow rate: 1.11 L/min


In [1]:
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s
print(f'Mean water flow rate: {MWF*enex.m32L/enex.s2m:.3} L/min')

EB       = enex.ElectricBoiler()
EB.k_shell = 25
EB.r0    = 0.2
EB.H     = 0.8
EB.x_ins = 0.05
EB.dV_w_serv = MWF
EB.system_update()

# ----------------------------
# 데이터 정의 (Energy, Entropy, Exergy 각각 Tank + Mixing 포함)
# ----------------------------
# 데이터 정의 및 딕셔너리 구성
Energy = {
    "components": [
        {"value": EB.Q_w_sup_tank, "sign": 1, "label": 'Supply water\nto tank'},
        {"value": EB.E_heater, "sign": 1, "label": 'Electricity input\nto heater'},
        {"value": 0, "sign":  1, "label": ''},
        {"value": EB.Q_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": EB.Q_w_sup_mix, "sign":  1, "label": 'Supply water\nto mixing valve'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": EB.Q_w_serv, "sign": 1, "label": 'Served\nhot water'} 
    ],
    "color" : 'tw.lime:',
    "ylabel": 'Energy [W]',
    "ymax"   : 3600,
    "yint"   : 900
}
    
Entropy = {
    "components": [
        {"value": EB.S_w_sup_tank, "sign": 1, "label": 'Supply water\nto tank'},
        {"value": EB.S_heater, "sign": 1, "label": 'Electricity input\nto heater'},
        {"value": EB.S_g_tank, "sign": 1, "label": 'Consumption\nin tank'},
        {"value": EB.S_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": EB.S_w_sup_mix, "sign": 1, "label": 'Supply water\nto mixing valve'},
        {"value": EB.S_g_mix, "sign": 1, "label": 'generated (mix)'},
        {"value": EB.S_w_serv, "sign": 1, "label": 'Served\nhot water'}
    ],
    "color" : 'tw.amber:',
    "ylabel": 'Entropy [W/K]',
    "ymax"   : 16,
    "yint"   : 4
}
    
Exergy = {
    "components": [
        {"value": EB.X_w_sup_tank, "sign": 1, "label": 'Supply water\nto tank'},
        {"value": EB.X_heater, "sign": 1, "label": 'Electricity input\nto heater'},
        {"value": EB.X_c_tank, "sign": -1, "label": 'Consumption\nin tank'},
        {"value": EB.X_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": EB.X_w_sup_mix, "sign": 1, "label": 'Supply water\nto mixing valve'},
        {"value": EB.X_c_mix, "sign": -1, "label": 'Consumption\nin mixing valve'},
        {"value": EB.X_w_serv, "sign": 1, "label": 'Served\nhot water'}
    ],
    "color" : 'tw.purple:',
    "ylabel":  'Exergy [W]',
    "ymax"   : 3600,
    "yint"   : 900
}

data_list = [Energy, Exergy]

# Figure 구성
nrows = 3
ncols = 1

fig, axs = plt.subplots(2, 1, figsize=(dm.cm2in(14), dm.cm2in(8)), dpi=200)

for i, data in enumerate(data_list):
    
    ax = axs[i]
    
    # Components
    components = data["components"]
    value      = np.array([comp["value"] for comp in components])
    sign       = np.array([comp["sign"] for comp in components])
    labels     = [comp["label"] for comp in components]
    signed     = value * sign
    color      = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.05  
    bm = DL * 0.06 
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops    = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1]  = last_height

    # bar plot
    bars = ax.bar(
        x_pos, signed, width=bw, bottom=bottoms,
        color=[color + '500' if h > 0 else color + '100' for h in signed],
        edgecolor=[color + '800']*len(signed),
        linewidth=0.5
    )
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        text_y = bottoms[j] + h + offset if h >= 0 else tops[j] - h + offset
        ax.text(bar.get_x() + bw/2, text_y, '' if h == 0 else f'{h:.0f}',
            ha='center', va='bottom', fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-1))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize = dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(min(0, min(value)*1.1), data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(0), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.065, 0.5)
    
# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=(0, 1, 0, 1))
plt.savefig('figure/EB_waterfall.svg')
plt.savefig('figure/EB_waterfall.png', dpi=600)
dm.util.save_and_show(fig)

NameError: name 'enex' is not defined

In [9]:
enex.print_balance(EB.exergy_balance)



HOT WATER TANK EXERGY BALANCE:====================

IN ENTRIES:
E_heater: 2762.73 [W]
X_w_sup_tank: 9.7 [W]

OUT ENTRIES:
X_w_tank: 312.56 [W]
X_l_tank: 8.93 [W]

CON ENTRIES:
X_c_tank: 2450.93 [W]


MIXING VALVE EXERGY BALANCE:======================

IN ENTRIES:
X_w_tank: 312.56 [W]
X_w_sup_mix: 4.16 [W]

OUT ENTRIES:
X_w_serv: 259.23 [W]

CON ENTRIES:
X_c_mix: 57.49 [W]


## GB

In [3]:
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s
print(f'Mean water flow rate: {MWF*enex.m32L/enex.s2m:.3} L/min')

GB       = enex.GasBoiler()
GB.r0    = 0.12
GB.H     = 0.45
GB.x_ins = 0.05
GB.dV_w_serv = MWF
GB.system_update()

# ----------------------------
# 데이터 정의 (Energy, Entropy, Exergy 각각 Tank + Mixing 포함)
# ----------------------------
# 데이터 정의 및 딕셔너리 구성
Energy = {
    "components": [
        {"value": GB.Q_w_sup, "sign": 1, "label": 'Supply water\nto chamber'},
        {"value": GB.E_NG, "sign": 1, "label": 'Gas input\nto chamber'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": GB.Q_exh, "sign": -1, "label": 'Exhaust gas\nheat loss'},
        {"value": 0, "sign":  1, "label": ''},
        {"value": GB.Q_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": GB.Q_w_sup_mix, "sign":  1, "label": 'Supply water\nto mixer'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": GB.Q_w_serv, "sign": 1, "label": 'Served\nhot water'} 
    ],
    "color" : 'tw.lime:',
    "ylabel": 'Energy [W]',
    "ymax"   : 4000,
    "yint"   : 1000
}
    
Entropy = {
    "components": [
        {"value": GB.S_w_sup, "sign": 1, "label": 'Supply water\nto chamber'},
        {"value": GB.S_NG, "sign": 1, "label": 'Gas input\nto chamber'},
        {"value": 0, "sign": 1, "label": 'Generation\nin chamber'},
        {"value": GB.S_g_tank, "sign": 1, "label": 'Consumption\nin tank'},
        {"value": 0, "sign": 1, "label": 'Generation\nin tank'},
        {"value": GB.S_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": GB.S_w_sup_mix, "sign": 1, "label": 'Supply water\nto mixer'},
        {"value": GB.S_g_mix, "sign": 1, "label": 'Generation\n in mixer'},
        {"value": GB.S_w_serv, "sign": 1, "label": 'Served\nhot water'}
    ],
    "color" : 'tw.amber:',
    "ylabel": 'Entropy [W/K]',
    "ymax"   : 16,
    "yint"   : 4
}
    
Exergy = {
    "components": [
        {"value": GB.X_w_sup, "sign": 1, "label": 'Supply water\nto chamber'},
        {"value": GB.X_NG, "sign": 1, "label": 'Gas input\nto chamber'},
        {"value": GB.X_c_comb, "sign": -1, "label": 'Consumption\nin chamber'},
        {"value": GB.X_exh, "sign": -1, "label": 'Exhaust gas\nheat loss'},
        {"value": GB.X_c_tank, "sign": -1, "label": 'Consumption\nin tank'},
        {"value": GB.X_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": GB.X_w_sup_mix, "sign": 1, "label": 'Supply water\nto mixer'},
        {"value": GB.X_c_mix, "sign": -1, "label": 'Consumption\nin mixer'},
        {"value": GB.X_w_serv, "sign": 1, "label": 'Served\nhot water'}
    ],
    "color" : 'tw.purple:',
    "ylabel":  'Exergy [W]',
    "ymax"   : 3000,
    "yint"   : 1000
}

data_list = [Energy, Exergy]

# Figure 구성
nrows = 3
ncols = 1

fig, axs = plt.subplots(2, 1, figsize=(dm.cm2in(16), dm.cm2in(8)), dpi=200)

for i, data in enumerate(data_list):
    
    ax = axs[i]
    
    # Components
    components = data["components"]
    value      = np.array([comp["value"] for comp in components])
    sign       = np.array([comp["sign"] for comp in components])
    labels     = [comp["label"] for comp in components]
    signed     = value * sign
    color      = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.05  
    bm = DL * 0.05  
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops    = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1]  = last_height

    # bar plot
    bars = ax.bar(
        x_pos, signed, width=bw, bottom=bottoms,
        color=[color + '500' if h > 0 else color + '100' for h in signed],
        edgecolor=[color + '800']*len(signed),
        linewidth=0.5
    )
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        text_y = bottoms[j] + h + offset if h >= 0 else tops[j] - h + offset
        ax.text(bar.get_x() + bw/2, text_y, '' if h == 0 else f'{h:.0f}',
            ha='center', va='bottom', fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-2))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize = dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(min(0, min(value)*1.1), data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(1), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.065, 0.5)
    
# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=(0, 1, 0, 1))
plt.savefig('figure/GB_waterfall.svg')
plt.savefig('figure/GB_waterfall.png', dpi=600)
dm.util.save_and_show(fig)


Mean water flow rate: 1.11 L/min


In [6]:
enex.print_balance(GB.exergy_balance)



COMBUSTION CHAMBER EXERGY BALANCE:================

IN ENTRIES:
X_NG: 2822.44 [W]
X_w_sup: 9.7 [W]

OUT ENTRIES:
X_w_comb_out: 315.86 [W]
X_exh: 61.91 [W]

CON ENTRIES:
X_c_comb: 2454.38 [W]


HOT WATER TANK EXERGY BALANCE:====================

IN ENTRIES:
X_w_comb_out: 315.86 [W]

OUT ENTRIES:
X_w_tank: 312.56 [W]
X_l_tank: 3.29 [W]

CON ENTRIES:
X_c_tank: 0.01 [W]


MIXING VALVE EXERGY BALANCE:======================

IN ENTRIES:
X_w_tank: 312.56 [W]
X_w_sup_mix: 4.16 [W]

OUT ENTRIES:
X_w_serv: 259.23 [W]

CON ENTRIES:
X_c_mix: 57.49 [W]


In [11]:
enex.print_balance(GB.exergy_balance)



COMBUSTION CHAMBER EXERGY BALANCE:================

IN ENTRIES:
X_NG: 2822.44 [W]
X_w_sup: 9.7 [W]

OUT ENTRIES:
X_w_comb_out: 315.86 [W]
X_exh: 61.91 [W]

CON ENTRIES:
X_c_comb: 2454.38 [W]


HOT WATER TANK EXERGY BALANCE:====================

IN ENTRIES:
X_w_comb_out: 315.86 [W]

OUT ENTRIES:
X_w_tank: 312.56 [W]
X_l_tank: 3.29 [W]

CON ENTRIES:
X_c_tank: 0.01 [W]


MIXING VALVE EXERGY BALANCE:======================

IN ENTRIES:
X_w_tank: 312.56 [W]
X_w_sup_mix: 4.16 [W]

OUT ENTRIES:
X_w_serv: 259.23 [W]

CON ENTRIES:
X_c_mix: 57.49 [W]


## HPB

In [8]:
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s
print(f'Mean water flow rate: {MWF*enex.m32L/enex.s2m:.3} L/min')

HPB       = enex.HeatPumpBoiler()
HPB.r0    = 0.2
HPB.H     = 0.8
HPB.x_ins = 0.05
HPB.dV_w_serv = MWF
HPB.system_update()


Energy = {
    "components": [
        {"value": 0, "sign": 1, "label": ''},
        {"value": HPB.Q_r_ext, "sign": 1, "label": 'Heat transfer\nfrom air to\next unit ref'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": 0, "sign": -1, "label": ''},
        {"value": 0, "sign":  1, "label": ''},
        {"value": HPB.E_cmp, "sign": 1, "label": 'Elec input\nto comp'},
        {"value": 0, "sign": -1, "label": ''},
        {"value": 0, "sign": 1, "label": ''},
        {"value": HPB.Q_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": HPB.Q_w_sup_tank, "sign":  1, "label": 'Supply water\nto tank'},
        {"value": HPB.Q_w_serv, "sign": 1, "label": 'Served\nhot water'} 
    ],
    "color" : 'tw.lime:',
    "ylabel": 'Energy [W]',
    "ymax"   : 3600,
    "yint"   : 900
}
    
Entropy = {
    "components": [
        {"value": HPB.S_fan, "sign": 1, "label": 'Elec input\nto fan'},
        {"value": HPB.S_r_ext, "sign": 1, "label": 'Heat transfer\nfrom air to\next unit ref'},
        {"value": HPB.S_a_ext_in, "sign": 1, "label": 'Inlet air\nto ext unit'},
        {"value": HPB.S_g_ext, "sign": -1, "label": 'Generation\next unit'},
        {"value": HPB.S_a_ext_out, "sign":  1, "label": 'Exhaust air\nfrom ext unit'},
        {"value": HPB.S_cmp, "sign": -1, "label": 'Elec input\nto comp'},
        {"value": HPB.S_g_r, "sign": -1, "label": 'Generation\nin ref loop'},
        {"value": HPB.S_r_ext, "sign": -1, "label": 'Ext unit ref\nto air'},
        {"value": HPB.S_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": HPB.S_g_tank, "sign": 1, "label": 'Generation\nin tank'},
        {"value": HPB.S_w_sup_tank, "sign":  1, "label": 'Supply water\nto tank'},
        {"value": HPB.S_w_serv, "sign": 1, "label": 'Served\nhot water'}
    ],
    "color" : 'tw.amber:',
    "ylabel": 'Entropy [W/K]',
    "ymax"   : 16,
    "yint"   : 4
}
    
Exergy = {
    "components": [
        {"value": HPB.X_fan, "sign": 1, "label": 'Elec input\nto fan'},
        {"value": HPB.X_r_ext, "sign": 1, "label": 'Heat transfer\nfrom air to\next unit ref'},
        {"value": HPB.X_a_ext_in, "sign": 1, "label": 'Inlet air\nto ext unit'},
        {"value": HPB.X_c_ext, "sign": -1, "label": 'Consumption\next unit'},
        {"value": HPB.X_a_ext_out, "sign":  -1, "label": 'Exhaust air\nfrom ext unit'},
        {"value": HPB.X_cmp, "sign": 1, "label": 'Elec input\nto comp'},
        {"value": HPB.X_c_r, "sign": -1, "label": 'Consumption\nin ref loop'},
        {"value": HPB.X_r_ext, "sign": -1, "label": 'Ext unit ref\nto air'},
        {"value": HPB.X_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": HPB.X_c_tank, "sign": -1, "label": 'Consumption\nin tank'},
        {"value": HPB.X_w_sup_tank, "sign":  1, "label": 'Supply water\nto tank'},
        {"value": HPB.X_w_serv, "sign": 1, "label": 'Served\nhot water'} 
    ],
    "color" : 'tw.purple:',
    "ylabel": 'Exergy [W]',
    "ymax"   : 3600,
    "yint"   : 900
}

data_list = [Energy, Exergy]

# Figure 구성
nrows = 2
ncols = 1

fig, axs = plt.subplots(2, 1, figsize=(dm.cm2in(18), dm.cm2in(8)), dpi=200)

for i, data in enumerate(data_list):
    
    ax = axs[i]
    
    # Components
    components = data["components"]
    value      = np.array([comp["value"] for comp in components])
    sign       = np.array([comp["sign"] for comp in components])
    labels     = [comp["label"] for comp in components]
    signed     = value * sign
    color      = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.035  
    bm = DL * 0.05  
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops    = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1]  = last_height

    # bar plot
    bars = ax.bar(
        x_pos, signed, width=bw, bottom=bottoms,
        color=[color + '500' if h > 0 else color + '100' for h in signed],
        edgecolor=[color + '800']*len(signed),
        linewidth=0.5
    )
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        text_y = bottoms[j] + h + offset if h >= 0 else tops[j] - h + offset
        ax.text(bar.get_x() + bw/2, text_y, '' if h == 0 else f'{h:.0f}',
            ha='center', va='bottom', fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-2))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize = dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(0, data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(1), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.065, 0.5)
    
# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=(0, 1, 0, 1))
plt.savefig('figure/HPB_waterfall.svg')
plt.savefig('figure/HPB_waterfall.png', dpi=600)
dm.util.save_and_show(fig)

Mean water flow rate: 1.11 L/min


In [10]:
enex.print_balance(HPB.exergy_balance)



EXTERNAL UNIT EXERGY BALANCE:=====================

IN ENTRIES:
E_fan: 85.15 [W]
X_r_ext: 62.99 [W]
X_a_ext_in: 0.0 [W]

CON ENTRIES:
X_c_ext: 133.57 [W]

OUT ENTRIES:
X_a_ext_out: 14.57 [W]


REFRIGERANT LOOP EXERGY BALANCE:==================

IN ENTRIES:
E_cmp: 1105.09 [W]

CON ENTRIES:
X_c_r: 511.04 [W]

OUT ENTRIES:
X_r_tank: 531.06 [W]
X_r_ext: 62.99 [W]


HOT WATER TANK EXERGY BALANCE:====================

IN ENTRIES:
X_r_tank: 531.06 [W]
X_w_sup_tank: 9.7 [W]

CON ENTRIES:
X_c_tank: 219.26 [W]

OUT ENTRIES:
X_w_tank: 312.56 [W]
X_l_tank: 8.93 [W]


MIXING VALVE EXERGY BALANCE:======================

IN ENTRIES:
X_w_tank: 312.56 [W]
X_w_sup_mix: 4.16 [W]

CON ENTRIES:
X_c_mix: 57.49 [W]

OUT ENTRIES:
X_w_serv: 259.23 [W]


## SHW

In [2]:
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s
# print(f'Mean water flow rate: {MWF*enex.m32L/enex.s2m:.3} L/min')   
SHW = enex.SolarHotWater()
SHW.dV_w_serv = MWF
SHW.I_DN = 500
SHW.I_dH = 200
SHW.A_stp = 4
SHW.system_update()
enex.print_balance(SHW.exergy_balance,0)
print(f"{SHW.S_dH/SHW.S_DN}")



SOLAR THERMAL PANEL EXERGY BALANCE:===============

IN ENTRIES:
X_sol: 2581.0 [W]
X_w_sup: 10.0 [W]

CON ENTRIES:
X_c_stp: 2119.0 [W]

OUT ENTRIES:
X_w_stp_out: 91.0 [W]
X_l: 381.0 [W]


COMBUSTION CHAMBER EXERGY BALANCE:================

IN ENTRIES:
X_w_stp_out: 91.0 [W]
X_NG: 1606.0 [W]

CON ENTRIES:
X_c_comb: 1349.0 [W]

OUT ENTRIES:
X_exh: 35.0 [W]
X_w_comb: 313.0 [W]


MIXING VALVE EXERGY BALANCE:======================

IN ENTRIES:
X_w_comb: 313.0 [W]
X_w_sup_mix: 4.0 [W]

CON ENTRIES:
X_c_mix: 57.0 [W]

OUT ENTRIES:
X_w_serv: 259.0 [W]
1.3284342138002634


In [107]:
SAGB = enex.SolarAssistedGasBoiler()
SAGB.dV_w_serv = 1.0
SAGB.I_DN = 500
SAGB.I_dH = 200
SAGB.x_ins = 0.05
SAGB.A_stp = 2
SAGB.system_update()

# ----------------------------
# 데이터 정의 (Energy, Entropy, Exergy 각각 Tank + Mixing 포함)
# ----------------------------
# 데이터 정의 및 딕셔너리 구성


Energy = {
    "components": [
        {"value": SAGB.Q_w_sup, "sign": 1, "label": 'Supply water\nto solar panel'},
        {"value": SAGB.Q_sol, "sign": 1, "label": 'Solar radiation\nto solar panel'},
        {"value": 0, "sign":  1, "label": ''},
        {"value": SAGB.Q_l, "sign": -1, "label": 'Heat loss\nfrom solar panel'},
        {"value": SAGB.E_NG, "sign": 1, "label": 'Gas input\nto chamber'},
        {"value": 0, "sign": -1, "label": ''},
        {"value": SAGB.Q_exh, "sign": -1, "label": 'Exhaust gas\nheat loss'},
        {"value": SAGB.Q_w_sup_mix, "sign":  1, "label": 'Supply water\nto mixing valve'},
        {"value": 0, "sign": 1, "label": ''},
        {"value": SAGB.Q_w_serv, "sign": 1, "label": 'Served\nhot water'} 
    ],
    "color" : 'tw.lime:',
    "ylabel": 'Energy [W]',
    "ymax"   : 3200,
    "yint"   : 800
}
    
Exergy = {
    "components": [
        {"value": SAGB.X_w_sup, "sign": 1, "label": 'Supply water\nto solar panel'},
        {"value": SAGB.X_sol, "sign": 1, "label": 'Solar radiation\nto solar panel'},
        {"value": SAGB.X_c_stp, "sign": -1, "label": 'Consumption\nin solar panel'},
        {"value": SAGB.X_l, "sign": -1, "label": 'Heat loss\nfrom solar panel'},
        {"value": SAGB.X_NG, "sign": 1, "label": 'Gas input\nto chamber'},
        {"value": SAGB.X_c_comb, "sign": -1, "label": 'Consumption\nin chamber'},
        {"value": SAGB.X_exh, "sign": -1, "label": 'Exhaust gas\nheat loss'},
        {"value": SAGB.X_w_sup_mix, "sign": 1, "label": 'Supply water\nto mixing valve'},
        {"value": SAGB.X_c_mix, "sign": -1, "label": 'Consumption\nin mixing valve'},
        {"value": SAGB.X_w_serv, "sign": 1, "label": 'Served\nhot water'}
    ],
    "color" : 'tw.purple:',
    "ylabel":  'Exergy [W]',
    "ymax"   : 3200,
    "yint"   : 800
}

data_list = [Energy, Exergy]

# Figure 구성
nrows = 2
ncols = 1

fig, axs = plt.subplots(nrows, ncols, figsize=(dm.cm2in(20), dm.cm2in(8)), dpi=200)

for i, data in enumerate(data_list):
    
    ax = axs[i]
    
    # Components
    components = data["components"]
    value      = np.array([comp["value"] for comp in components])
    sign       = np.array([comp["sign"] for comp in components])
    labels     = [comp["label"] for comp in components]
    signed     = value * sign
    color      = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.03  
    bm = DL * 0.04 
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops    = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1]  = last_height

    # bar plot
    bars = ax.bar(
        x_pos, signed, width=bw, bottom=bottoms,
        color=[color + '500' if h > 0 else color + '100' for h in signed],
        edgecolor=[color + '800' if h > 0 else color + '500' for h in signed],
        linewidth=0.5
    )
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        text_y = bottoms[j] + h + offset if h >= 0 else tops[j] - h + offset
        ax.text(bar.get_x() + bw/2, text_y, '' if h == 0 else f'{h:.0f}',
            ha='center', va='bottom', fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-1))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize = dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(min(0, min(value)*1.1), data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(0), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.05, 0.5)
    
# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=(0, 1, 0, 1))
plt.savefig('figure/waterfall/SAGB_waterfall.svg', transparent=True)
plt.savefig('figure/waterfall/SAGB_waterfall.png',dpi=600)
dm.util.save_and_show(fig)

In [105]:
enex.print_balance(SAGB.energy_balance, 0)



SOLAR THERMAL PANEL ENERGY BALANCE:===============

IN ENTRIES:
Q_sol: 1330.0 [W]
Q_w_sup: 488.0 [W]

OUT ENTRIES:
Q_w_stp_out: 1087.0 [W]
Q_l: 731.0 [W]


COMBUSTION CHAMBER ENERGY BALANCE:================

IN ENTRIES:
Q_w_stp_out: 1087.0 [W]
E_NG: 2048.0 [W]

OUT ENTRIES:
Q_exh: 205.0 [W]
Q_w_comb: 2930.0 [W]


MIXING VALVE ENERGY BALANCE:======================

IN ENTRIES:
Q_w_comb: 2930.0 [W]
Q_w_sup_mix: 209.0 [W]

OUT ENTRIES:
Q_w_serv: 3140.0 [W]


## GSHPB

In [19]:

# GSHPB update
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s

GSHPB       = enex.GroundSourceHeatPumpBoiler()
GSHPB.r0    = 0.2
GSHPB.H     = 0.8
GSHPB.x_ins = 0.10
GSHPB.dV_w_serv = MWF
GSHPB.system_update()

    
Exergy = {
    "components": [
        {"value": GSHPB.Xin_g, "sign": 1, "label": 'Exergy from\n ground'},
        {"value": GSHPB.Xc_g, "sign": -1, "label": 'Consumption\nin ground'},
        {"value": GSHPB.E_pmp, "sign": 1, "label": 'Elec input\nto pump'},
        {"value": GSHPB.Xc_GHE, "sign": -1, "label": 'Consumption\n in GHE'},
        {"value": GSHPB.Xc_ext, "sign":  -1, "label": 'Consumption\n in ext unit'},
        {"value": GSHPB.X_cmp, "sign": 1, "label": 'Elec input\nto comp'},
        {"value": GSHPB.Xc_r, "sign": -1, "label": 'Consumption\nin ref loop'},
        {"value": GSHPB.X_l_tank, "sign": -1, "label": 'Heat loss\nfrom tank'},
        {"value": GSHPB.X_w_sup_tank, "sign":  1, "label": 'Supply water\nto tank'},
        {"value": GSHPB.Xc_tank, "sign": -1, "label": 'Consumption\nin tank'},
        {"value": GSHPB.X_w_sup_mix, "sign":  1, "label": 'Supply water\nto mixer'},
        {"value": GSHPB.Xc_mix, "sign": -1, "label": 'Consumption\nin mixing valve'},
        {"value": GSHPB.X_w_serv, "sign": 1, "label": 'Served\nhot water'} 
    ],
    "color" : 'tw.purple:',
    "ylabel": 'Exergy [W]',
    "ymax"   : 800,
    "yint"   : 200,
}

data_list = [Exergy]

# Figure 구성

fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(18), dm.cm2in(5)), dpi=200)

for i, data in enumerate(data_list):
    
    # Components
    components = data["components"]
    value      = np.array([comp["value"] for comp in components])
    sign       = np.array([comp["sign"] for comp in components])
    labels     = [comp["label"] for comp in components]
    signed     = value * sign
    color      = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.035  
    bm = DL * 0.04  
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops    = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1]  = last_height

    # bar plot
    bars = ax.bar(
        x_pos, signed, width=bw, bottom=bottoms,
        color=[color + '500' if h > 0 else color + '100' for h in signed],
        edgecolor=[color + '800' if h > 0 else color + '500' for h in signed],
        linewidth=0.5
    )
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        text_y = bottoms[j] + h + offset if h >= 0 else tops[j] - h + offset
        ax.text(bar.get_x() + bw/2, text_y, '' if h == 0 else f'{h:.0f}',
            ha='center', va='bottom', fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-2))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize = dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(0, data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(1), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.05, 0.5)
    
# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=(0, 1, 0, 1))
plt.savefig('figure/GSHPB_waterfall.svg')
plt.savefig('figure/GSHPB_waterfall.png', dpi=600)
dm.util.save_and_show(fig)

## GSHP_cooling

In [18]:

# GSHPc update
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s

GSHPc       = enex.GroundSourceHeatPump_cooling()
GSHPc.Q_r_int = 10000 # W
GSHPc.system_update()

    
Exergy = {
    "components": [
        {"value": GSHPc.Xin_g, "sign": 1, "label": 'Exergy from\n undisturb ground'},
        {"value": GSHPc.Xc_g, "sign": -1, "label": 'Consumption\nin ground'},
        {"value": GSHPc.E_pmp, "sign": 1, "label": 'Elec input\nto pump'},
        {"value": GSHPc.Xc_GHE, "sign": -1, "label": 'Consumption\n in GHE'},
        {"value": GSHPc.Xc_exch, "sign":  -1, "label": 'Consumption\n in heat exchanger'},
        {"value": GSHPc.E_cmp, "sign": 1, "label": 'Elec input\nto compressor'},
        {"value": GSHPc.Xc_r, "sign": -1, "label": 'Consumption\nin ref loop'},
        {"value": GSHPc.E_fan_int, "sign": 1, "label": 'Elec input\nto fan'},
        {"value": GSHPc.X_a_int_in, "sign": 1, "label": 'Exergy from\nroom air'},
        {"value": GSHPc.Xc_int, "sign": -1, "label": 'Consumption\nin internal unit'},
        {"value": GSHPc.Xout_int, "sign": 1, "label": 'Supply air\nto room'} 
    ],
    "color" : 'tw.purple:',
    "ylabel": 'Exergy [W]',
    "ymax"   : 1000,
    "yint"   : 200,
}

data_list = [Exergy]

# Figure 구성

fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(18), dm.cm2in(5)), dpi=200)

for i, data in enumerate(data_list):
    
    # Components
    components = data["components"]
    value      = np.array([comp["value"] for comp in components])
    sign       = np.array([comp["sign"] for comp in components])
    labels     = [comp["label"] for comp in components]
    signed     = value * sign
    color      = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.035  
    bm = DL * 0.04  
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops    = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1]  = last_height

    # bar plot
    bars = ax.bar(
        x_pos, signed, width=bw, bottom=bottoms,
        color=[color + '500' if h > 0 else color + '100' for h in signed],
        edgecolor=[color + '800' if h > 0 else color + '500' for h in signed],
        linewidth=0.5
    )
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        text_y = bottoms[j] + h + offset if h >= 0 else tops[j] - h + offset
        ax.text(bar.get_x() + bw/2, text_y, '' if h == 0 else f'{h:.0f}',
            ha='center', va='bottom', fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-2))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize = dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(0, data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(1), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.05, 0.5)
    
# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=(0, 1, 0, 1))
plt.savefig('figure/GSHPc_waterfall.svg')
plt.savefig('figure/GSHPc_waterfall.png',dpi=600)
dm.util.save_and_show(fig)

In [19]:
enex.print_balance(GSHPc.exergy_balance, 0)



INTERNAL UNIT EXERGY BALANCE:=====================

IN ENTRIES:
$X_{f,int}$: 233.0 [W]
$X_{r,int}$: 899.0 [W]
$X_{a,int,in}$: 169.0 [W]

CON ENTRIES:
$X_{c,int}$: 610.0 [W]

OUT ENTRIES:
$X_{a,int,out}$: 690.0 [W]


REFRIGERANT LOOP EXERGY BALANCE:==================

IN ENTRIES:
$X_{cmp}$: 1761.0 [W]
$X_{r,exch}$: -233.0 [W]

CON ENTRIES:
$X_{c,r}$: 629.0 [W]

OUT ENTRIES:
$X_{r,int}$: 899.0 [W]


HEAT EXCHANGER EXERGY BALANCE:====================

IN ENTRIES:
$X_{f,out}$: 94.0 [W]

CON ENTRIES:
$X_{c,exch}$: 323.0 [W]

OUT ENTRIES:
$X_{r,exch}$: -233.0 [W]
$X_{f,in}$: 3.0 [W]


GROUND HEAT EXCHANGER EXERGY BALANCE:=============

IN ENTRIES:
$E_{pmp}$: 200 [W]
$X_{b}$: 337.0 [W]
$X_{f,in}$: 3.0 [W]

CON ENTRIES:
$X_{c,GHE}$: 446.0 [W]

OUT ENTRIES:
$X_{f,out}$: 94.0 [W]


GROUND EXERGY BALANCE:============================

IN ENTRIES:
$X_{g}$: 602.0 [W]

CON ENTRIES:
$X_{c,g}$: 265.0 [W]

OUT ENTRIES:
$X_{b}$: 337.0 [W]


In [20]:
ASHPc = enex.AirSourceHeatPump_cooling()
ASHPc.Q_r_int = 10000 # W
ASHPc.system_update()
enex.print_balance(ASHPc.exergy_balance, 0)



INTERNAL UNIT EXERGY BALANCE:=====================

IN ENTRIES:
$E_{f,int}$: 233.0 [W]
$X_{r,int}$: 899.0 [W]

CON ENTRIES:
$X_{c,int}$: 610.0 [W]

OUT ENTRIES:
$X_{a,int,out}$: 690.0 [W]
$X_{a,int,in}$: 169.0 [W]


REFRIGERANT LOOP EXERGY BALANCE:==================

IN ENTRIES:
$E_{cmp}$: 2156.0 [W]

CON ENTRIES:
$X_{c,r}$: 684.0 [W]

OUT ENTRIES:
$X_{r,int}$: 899.0 [W]
$X_{r,ext}$: 573.0 [W]


EXTERNAL UNIT EXERGY BALANCE:=====================

IN ENTRIES:
$E_{f,ext}$: 261.0 [W]
$X_{r,ext}$: 573.0 [W]

CON ENTRIES:
$X_{c,ext}$: 638.0 [W]

OUT ENTRIES:
$X_{a,ext,out}$: 196.0 [W]
$X_{a,ext,in}$: 0.0 [W]


In [9]:
GSHPc = enex.GroundSourceHeatPump_cooling()
GSHPc.T0 = 32.0
GSHPc.T_g = 19.0
GSHPc.T_a_room = 20.0
GSHPc.T_a_int_out = 14.0
GSHPc.Q_r_int = 10000.0
GSHPc.T_r_int = 9.0
GSHPc.T_r_exch = 29.0
GSHPc.D = 150.0
GSHPc.H = 100.0
GSHPc.r_b = 0.075
GSHPc.R_b = 0.1
GSHPc.V_f = 24.0/60000
GSHPc.E_pmp = 200.0
GSHPc.k_g = 2.0
GSHPc.c_g = 800.0
GSHPc.rho_g = 2000.0
GSHPc.system_update()

enex.print_balance(GSHPc.exergy_balance, 0)



INTERNAL UNIT EXERGY BALANCE:=====================

IN ENTRIES:
$X_{f,int}$: 358.0 [W]
$X_{r,int}$: 815.0 [W]
$X_{a,int,in}$: 404.0 [W]

CON ENTRIES:
$X_{c,int}$: 656.0 [W]

OUT ENTRIES:
$X_{a,int,out}$: 921.0 [W]


REFRIGERANT LOOP EXERGY BALANCE:==================

IN ENTRIES:
$X_{cmp}$: 1434.0 [W]
$X_{r,exch}$: 114.0 [W]

CON ENTRIES:
$X_{c,r}$: 733.0 [W]

OUT ENTRIES:
$X_{r,int}$: 815.0 [W]


HEAT EXCHANGER EXERGY BALANCE:====================

IN ENTRIES:
$X_{f,out}$: 145.0 [W]

CON ENTRIES:
$X_{c,exch}$: 31.0 [W]

OUT ENTRIES:
$X_{r,exch}$: 114.0 [W]
$X_{f,in}$: 1.0 [W]


GROUND HEAT EXCHANGER EXERGY BALANCE:=============

IN ENTRIES:
$E_{pmp}$: 200.0 [W]
$X_{b}$: 360.0 [W]
$X_{f,in}$: 1.0 [W]

CON ENTRIES:
$X_{c,GHE}$: 416.0 [W]

OUT ENTRIES:
$X_{f,out}$: 145.0 [W]


GROUND EXERGY BALANCE:============================

IN ENTRIES:
$X_{g}$: 500.0 [W]

CON ENTRIES:
$X_{c,g}$: 140.0 [W]

OUT ENTRIES:
$X_{b}$: 360.0 [W]


## GSHP_heating

In [18]:
# GSHPh update
water_use_in_a_day = 0.2 # m3/day
water_use_hour = 3 # h
MWF = water_use_in_a_day / (water_use_hour*enex.h2s) # m3/s

GSHPh       = enex.GroundSourceHeatPump_heating()
GSHPh.system_update()

Exergy = {
    "components": [
        {"value": GSHPh.Xin_g, "sign": 1, "label": 'Exergy from\n undisturb ground'},
        {"value": GSHPh.Xc_g, "sign": -1, "label": 'Consumption\nin ground'},
        {"value": GSHPh.E_pmp, "sign": 1, "label": 'Elec input\nto pump'},
        {"value": GSHPh.Xc_GHE, "sign": -1, "label": 'Consumption\n in GHE'},
        {"value": GSHPh.Xc_ext, "sign":  -1, "label": 'Consumption\n in external unit'},
        {"value": GSHPh.E_cmp, "sign": 1, "label": 'Elec input\nto compressor'},
        {"value": GSHPh.Xc_r, "sign": -1, "label": 'Consumption\nin ref loop'},
        {"value": GSHPh.E_fan_int, "sign": 1, "label": 'Elec input\nto fan'},
        {"value": GSHPh.X_a_int_in, "sign": 1, "label": 'Exergy from\nroom air'},
        {"value": GSHPh.Xc_int, "sign": -1, "label": 'Consumption\nin internal unit'},
        {"value": GSHPh.Xout_int, "sign": 1, "label": 'Supply air\nto room'} 
    ],
    "color" : 'tw.purple:',
    "ylabel": 'Exergy [W]',
    "ymax"   : 1000,
    "yint"   : 200,
}

data_list = [Exergy]

# Figure 구성

fig, ax = plt.subplots(1, 1, figsize=(dm.cm2in(18), dm.cm2in(5)), dpi=200)

for i, data in enumerate(data_list):
    
    # Components
    components = data["components"]
    value      = np.array([comp["value"] for comp in components])
    sign       = np.array([comp["sign"] for comp in components])
    labels     = [comp["label"] for comp in components]
    signed     = value * sign
    color      = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.035  
    bm = DL * 0.04  
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops    = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1]  = last_height

    # bar plot
    bars = ax.bar(
        x_pos, signed, width=bw, bottom=bottoms,
        color=[color + '500' if h > 0 else color + '100' for h in signed],
        edgecolor=[color + '800' if h > 0 else color + '500' for h in signed],
        linewidth=0.5
    )
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        text_y = bottoms[j] + h + offset if h >= 0 else tops[j] - h + offset
        ax.text(bar.get_x() + bw/2, text_y, '' if h == 0 else f'{h:.0f}',
            ha='center', va='bottom', fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-2))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize = dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(0, data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(1), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.05, 0.5)
    
# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=(0, 1, 0, 1))
plt.savefig('figure/GSHPh_waterfall.svg')
plt.savefig('figure/GSHPh_waterfall.png',dpi=600)
dm.util.save_and_show(fig)

In [6]:
enex.print_balance(GSHPh.exergy_balance,0)



INTERNAL UNIT EXERGY BALANCE:=====================

IN ENTRIES:
$X_{f,int}$: 132.0 [W]
$X_{r,int}$: 454.0 [W]
$X_{a,int,in}$: 279.0 [W]

CON ENTRIES:
$X_{c,int}$: 251.0 [W]

OUT ENTRIES:
$X_{a,int,out}$: 614.0 [W]


REFRIGERANT LOOP EXERGY BALANCE:==================

IN ENTRIES:
$X_{cmp}$: 692.0 [W]
$X_{r,ext}$: 59.0 [W]

CON ENTRIES:
$X_{c,r}$: 297.0 [W]

OUT ENTRIES:
$X_{r,int}$: 454.0 [W]


EXTERNAL UNIT EXERGY BALANCE:=====================

IN ENTRIES:
$X_{f,out}$: 527.0 [W]

CON ENTRIES:
$X_{c,ext}$: 75.0 [W]

OUT ENTRIES:
$X_{r,ext}$: 59.0 [W]
$X_{f,in}$: 392.0 [W]


GROUND HEAT EXCHANGER EXERGY BALANCE:=============

IN ENTRIES:
$E_{pmp}$: 200 [W]
$X_{b}$: 152.0 [W]
$X_{f,in}$: 392.0 [W]

CON ENTRIES:
$X_{c,GHE}$: 217.0 [W]

OUT ENTRIES:
$X_{f,out}$: 527.0 [W]


GROUND EXERGY BALANCE:============================

IN ENTRIES:
$X_{g}$: 162.0 [W]

CON ENTRIES:
$X_{c,g}$: 10.0 [W]

OUT ENTRIES:
$X_{b}$: 152.0 [W]


## EH

In [20]:
EH = enex.ElectricHeater()
EH.T0 = 0
EH.T_rs = 10
EH.D_p = 0.005 # 
EH.E_heater = 1000 # [W]
EH.W_p = 1 # [m3/s]
EH.dt = 10
EH.system_update()

Energy = {
    "components": [
        {"value": EH.E_heater, "sign": 1, "label": 'Electricity\ninput'},
        {"value": 0,   "sign": -1, "label": ''},
        {"value": EH.Q_rad_rs, "sign": 1, "label": 'Radiation\nfrom surrounding surface'},
        {"value": EH.Q_rad_hs, "sign": -1, "label": 'Radiation\nfrom panel surface'},
        {"value": 0,   "sign": -1, "label": ''},
        {"value": EH.Q_conv, "sign": -1, "label": 'Convection\nto air'},
    ],
    "color": 'tw.lime:',
    "ylabel": 'Energy [W]',
    "ymax": 1200,
    "yint": 300
}

Exergy = {
    "components": [
        {"value": EH.X_heater, "sign": 1, "label": 'Electricity\ninput'},
        {"value": EH.X_c_hb,   "sign": -1, "label": 'Consumption\nin heater body'},
        # {"value": EH.X_cond,   "sign": 1, "label": 'Heat transfer\nto surface'},
        {"value": EH.X_rad_rs, "sign": 1, "label": 'Radiation\nfrom surrounding surface'},
        {"value": EH.X_rad_hs, "sign": -1, "label": 'Radiation\nfrom panel surface'},
        {"value": EH.X_c_hs,  "sign": -1, "label": 'Consumption\nin radiation'},
        {"value": EH.X_conv, "sign": -1, "label": 'Convection\nto air'},
    ],
    "color": 'tw.purple:',
    "ylabel": 'Exergy [W]',
    "ymax": 1200,
    "yint": 300
}

data_list = [Energy, Exergy]

# Figure 구성
nrows = 2
ncols = 1

fig, axs = plt.subplots(nrows, ncols, figsize=(dm.cm2in(20), dm.cm2in(8)), dpi=200)

for i, data in enumerate(data_list):
    ax = axs[i]
    
    # Components
    components = data["components"]
    value = np.array([comp["value"] for comp in components])
    sign = np.array([comp["sign"] for comp in components])
    labels = [comp["label"] for comp in components]
    signed = value * sign
    color = data["color"]

    # Data length
    DL = len(value)
    
    # x position (bar position)
    x_pos = np.arange(DL)
    
    # bar width, bar margin
    bw = DL * 0.03  
    bm = DL * 0.04 
    
    # ----- 누적 하단 위치 계산 (Waterfall 구조) -----
    bottoms = np.concatenate([[0], np.cumsum(signed)[:-1]])
    tops = np.concatenate([bottoms[1:], [value[-1]]])  # 다음 막대의 bottom이 이전 막대의 top

    # 마지막 막대 재정의 (총합 혹은 served 등)
    if DL >= 2:
        last_height = bottoms[-1]
        bottoms[-1] = 0
        signed[-1] = last_height

    # bar plot
    bars = ax.bar(
        x_pos, signed, width=bw, bottom=bottoms,
        color=[color + '500' if h > 0 else color + '100' for h in signed],
        edgecolor=[color + '800' if h > 0 else color + '500' for h in signed],
        linewidth=0.5
    )
    
    # offset
    offset = data["ymax"] * 0.015
    
    # text
    for j, bar in enumerate(bars):
        h = bar.get_height()
        if h == 0:
            bar.set_edgecolor('none')
            bar.set_facecolor('none')
            continue
        text_y = bottoms[j] + h + offset if h >= 0 else tops[j] - h + offset
        ax.text(bar.get_x() + bw/2, text_y, '' if h == 0 else f'{h:.0f}',
                ha='center', va='bottom', fontsize=dm.fs(-2))

    for j in range(len(value) - 1):
        ax.hlines(y=tops[j], xmin=x_pos[j], xmax=x_pos[j+1],
                  color='tw.stone:500', lw=0.25, linestyle=':', zorder=-1)

    # set ticks
    ax.set_xticks(x_pos)
    ax.set_yticks(np.arange(0, data["ymax"] + data["yint"], data["yint"]))
    
    ax.set_xticklabels(labels, ha='center', fontweight=400, fontsize=dm.fs(-1))
    
    # tick parameters
    ax.tick_params(axis='x', length=0)
    ax.tick_params(axis='y', labelsize=dm.fs(-1))
    
    # set limits
    ax.set_xlim(-bm, (DL-1) + bm)
    ax.set_ylim(min(0, min(value)*1.1), data["ymax"])
    
    # set labels
    ax.set_ylabel(data["ylabel"], fontsize=dm.fs(0), fontweight=400, color='tw.stone:800')
    ax.yaxis.set_label_coords(-0.05, 0.5)
    
# 전체 레이아웃 및 저장
plt.subplots_adjust(hspace=0.6)
dm.simple_layout(fig, bbox=(0, 1, 0, 1))
plt.savefig('figure/EH_waterfall.svg')
plt.savefig('figure/EH_waterfall.png', dpi=600)
dm.util.save_and_show(fig)

# Test